In [1]:
cd ..

/Users/yu-lin.huang/git/statec/statnews


In [2]:
import ollama

In [17]:
from src.utils.env import INDICATOR_URL, SHEET_ANNUAL_VARIATION, SHEET_MENSUEL_VARIATION
from src.preprocess.ingestion import get_sheet, get_top_variations, get_target_index

In [18]:
def construire_prompt(target, variation, annual_variation, positives, negatives):
    return f"""
Tu es un assistant économique chargé d’écrire une courte analyse des prix alimentaires mensuels.

Le mois cible est : {target}

📊 Variation mensuelle : {variation}%  
📊 Variation annuelle : {annual_variation}%  
🔺 Produits en hausse : {positives}  
🔻 Produits en baisse : {negatives}

Rédige un paragraphe journalistique de 3 à 5 phrases.

Consignes :
- Commence par annoncer la variation globale (hausse ou baisse) en mensuelle.
- Mentionne le produit dont les prix a plus baissé (minimum) et deux autre produits dont les prix ont le plus baissé
- Mentionne le produit dont les prix a plus augmenté (maximum) et deux autre dont les prix ont le plus augmenté.
- Intègre les pourcentages entre parenthèses, avec - si négative.
- Utilise un ton neutre, fluide et informatif.
- Décrire juste la variation sans donner de commentaire ou conclusion.
- Terminer par annoncer la variation globale (hausse ou baisse) en annuel.
"""


In [19]:
df_mensuel = get_sheet(SHEET_MENSUEL_VARIATION)
df_annual = get_sheet(SHEET_ANNUAL_VARIATION)

In [20]:
target = 2024.06
alimentaire_mensuel_negative = get_top_variations(df_mensuel, target=target, prefix="01", positive=False, level='4', n=10)
alimentaire_mensuel_positive = get_top_variations(df_mensuel, target=target, prefix="01", positive=True, level='4', n=10)
alimentaire_mensuel_variation = get_target_index(df_mensuel, target=target, code='0101')
alimentaire_annuel_variation = get_target_index(df_annual, target=target, code='0101')

In [21]:
alimentaire_mensuel_negative

{'Nourriture pour bébés': -3.9,
 'Fruits frais ': -2.6,
 'Poisson surgelé': -2.3,
 'Fruits de mer surgelés': -1.9,
 'Légumes surgelés': -1.9,
 'Fruits surgelés': -1.8,
 'Oeufs': -1.8,
 'Céréales pour petit déjeuner': -1.7,
 'Autres graisses animales': -1.6,
 'Autres huiles alimentaires': -1.2}

In [22]:
prompt = construire_prompt(target, alimentaire_mensuel_variation, alimentaire_annuel_variation, alimentaire_mensuel_positive, alimentaire_mensuel_negative)

In [23]:
response = ollama.generate(
    model="qwen3:0.6b",
    prompt=prompt,
    options={
        'seed': 42
    }
)

In [24]:
print(response.response)

Le mois 2024 présente une baisse mensuelle (-0.4%) en raison des pertes de prix dans les produits comme les **Huile d'olive** (4.1%) et les **Fruits de mer frais** (3.2%), ainsi que dans les **Poisson et fruits de mer séchés** (1.0%) et les **Sel, épices et herbes culinaires** (0.9%). À la fin de l'année, le marché affiche une hausse annuelle (+1.6%) pour les **Boissons gazeuses** (1.3%) et les **Poissons surgelés** (2.3%), avec deux autres produits en hausse, comme les **Fruits de mer frais** (3.2%) et les **Fruits de mer surgelés** (1.9%).  

Le marché affiche une baisse mensuelle (-0.4%) en raison des pertes de prix dans les produits comme les **Nourriture pour bébés** (3.9%) et les **Céréales pour petit déjeuner** (1.7%).


In [11]:
alimentaire_mensuel_negative

{'Nourriture pour bébés': -3.9,
 'Fruits frais ': -2.6,
 'Poisson surgelé': -2.3,
 'Fruits de mer surgelés': -1.9,
 'Légumes surgelés': -1.9,
 'Fruits surgelés': -1.8,
 'Oeufs': -1.8,
 'Céréales pour petit déjeuner': -1.7,
 'Autres graisses animales': -1.6,
 'Autres huiles alimentaires': -1.2}

In [12]:
import requests

In [ ]:
token = ""

In [16]:

def chat_with_model(token, prompt):
    url = 'http://10.6.35.12:8080/ollama/api/generate'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    data = {
      "model": "mistral:7b",
      "prompt": prompt,
      "stream": False
    }
    response = requests.post(url, headers=headers, json=data)
    return response

In [17]:
response = chat_with_model(token, prompt)

In [18]:
print(response.json()['response'])

 Les prix des produits alimentaires ont baissé en moyenne de -0.4% en juin 2024. Parmi les produits, les fruits frais ont connu la plus forte baisse avec -2,6%, suivis par le poisson surgelé (-2,3%) et les légumes surgelés (-1,9%). En revanche, les prix des huiles alimentaires, notamment l'huile d'olive (+4,1%), ont augmenté en moyenne de 4,1%. Autres produits marqués par une augmentation des prix : les fruits de mer frais avec +3,2%, les chips et les boissons gazeuses non minérales (+1,3%) et le poisson et fruits de mer séchés, fumés ou salés (+1%). Globalement, les prix ont augmenté en moyenne de 1,6% au cours de l'année 2024.
